In [2]:
import json
from tqdm import tqdm_notebook as tqdm

In [43]:
SEQ_LEN = 64
BATCH_SIZE = 16

def get_batch(dataset):
    i = 0
    offset = 0
    while i < len(dataset):
        trains = []
        targets = []
        while len(trains) < BATCH_SIZE:
            
            sentense = dataset[i]
            train = torch.LongTensor(sentense[offset:offset+SEQ_LEN]).view(-1, 1)
            target = torch.LongTensor(sentense[offset + 1:offset + SEQ_LEN + 1]).view(-1, 1)
            trains.append(train)
            targets.append(target)
            
            offset+=1
            if offset + SEQ_LEN == len(sentense):
                offset = 0
                i += 1

        yield torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [44]:
trains, targets = next(get_batch(dataset))

In [19]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=50, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [2]:
# LSTM Model

In [20]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        # embeding chars to random vectors
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        # add LSTM layer
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        # shape: bach * sent_len * 1
        x = self.encoder(x).squeeze(2)
        # shape: bach * sent_len
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [33]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=64, embedding_size=8, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 500
loss_avg = []

for epoch in tqdm(range(n_epochs)):
    model.train()
    train, target = next(get_batch(dataset))
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 2.891374125480652
 тро  те  тсо  и  ття сте     тее тее тш    тек    
Loss: 1.496510820388794
 трясутся ит закк гремит так что стены трясутся итт
Loss: 0.3398489391803741
 гремит внешний блок гремит так что стены трясутся 
Loss: 0.10416097484529019
 гремит внешний блок гремит так что стены трясутся 
Loss: 0.0462927708029747
 гремит внешний блок гремит так что стены трясутся 
Loss: 0.027348949536681176
 гремит внешний блок гремит так что стены трясутся 
Loss: 0.019744685031473638
 гремит внешний блок гремит так что стены трясутся 
Loss: 0.01567565105855465
 гремит внешний блок гремит так что стены трясутся 
Loss: 0.012662233617156744
 гремит внешний блок гремит так что стены трясутся 
Loss: 0.010584806306287647
 гремит внешний блок гремит так что стены трясутся 


In [22]:
torch.save(model.state_dict(), "model1M")

In [36]:
start_text='в машине плохо'
prediction_len=200
temp=0.3

hidden = model.init_hidden()
idx_input = [char_to_idx[char] for char in start_text]
train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
predicted_text = start_text

_, hidden = model(train, hidden)

inp = train[-1].view(-1, 1, 1)

for i in range(prediction_len):
    output, hidden = model(inp.to(device), hidden)
    output_logits = output.cpu().data.view(-1)
    p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()    
    top_index = np.random.choice(len(char_to_idx), p=p_next)
    inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
    predicted_char = idx_to_char[top_index]
    predicted_text += predicted_char

print(predicted_text)

в машине плохотрясутся и это еще с закрытыми ото еще в ешний блок гремит так что стены трясутся и это еще с закрытыми ото еще в ешний блок гремит так что стены трясутся и это еще с закрытыми ото ещк сто стены трясу
